In [4]:
# 필요한 도구들을 컴퓨터에게 알려줍니다.
import pandas as pd  # 표 같은 데이터를 다루기 쉽게 해주는 도구입니다.
from sklearn.feature_extraction.text import CountVectorizer  # 문장에서 단어의 사용 빈도를 세어주는 도구입니다.
from konlpy.tag import Okt  # 한국어 문장을 분석해주는 도구입니다.

# 한국어 문장 분석을 위한 준비를 합니다.
okt = Okt()

# 엑셀 파일에서 데이터를 읽어와서 표 형태로 만듭니다.
data = pd.read_excel('빅카인즈_인공지능_뉴스.xlsx')  # Windows에서 경로 구분자는 \\ 를 사용합니다.

# 뉴스 제목과 본문을 합쳐서 새로운 열을 만듭니다.
data['제목_본문'] = data['제목'] + data['본문']

# 데이터를 가공하는 작업입니다. 여기서는 문장에서 명사와 형용사만 뽑아내는 작업을 합니다.
def preprocess_data(text):
    morphs = okt.pos(text)  # 문장을 분석합니다.
    # 명사와 형용사만 골라냅니다. 이전 코드에서의 오류를 수정합니다.
    words = [word for word, tag in morphs if tag in ['Noun', 'Adjective'] and len(word) > 1]
    return ' '.join(words)  # 골라낸 단어들을 하나의 문자열로 만듭니다.

# 모든 뉴스 데이터에 위에서 만든 가공 작업을 적용합니다.
data['preprocessing'] = data['제목_본문'].astype('str').apply(preprocess_data)

# 단어의 사용 빈도를 분석합니다.
cv = CountVectorizer(max_features=99999, ngram_range=(1, 1))  # 사용 빈도를 세는 도구를 설정합니다.
tdm = cv.fit_transform(data['preprocessing'])  # 가공된 데이터에서 각 단어의 사용 빈도를 세어 표로 만듭니다.
word_count_tf = pd.DataFrame({'단어': cv.get_feature_names_out(), '빈도': tdm.sum(axis=0).flat})  # 단어와 그 빈도를 표로 만듭니다.
word_count_tf = word_count_tf.sort_values('빈도', ascending=False)  # 빈도가 높은 순으로 정렬합니다.
word_count_tf = word_count_tf.reset_index(drop=True)  # 표의 인덱스를 재설정합니다.
word_count_tf.index = word_count_tf.index + 1  # 인덱스 번호를 1부터 시작하도록 조정합니다.

# 분석 결과를 파일로 저장합니다.
word_count_tf.to_csv('빈도분석.csv', encoding='cp949')  # 파일 이름에 사용한 경로 구분자나 추가 경로가 없으므로 수정할 필요 없습니다.
